# Library

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Crawling Data

Proses Crawling Data

In [ ]:
# Function to sanitize filename
def sanitize_filename(filename):
    # Replace invalid characters with underscore
    return re.sub(r'[<>:"/\\|?*]', '_', filename)

# Base URL for Putusan PN Jakarta Utara
base_url = "https://putusan3.mahkamahagung.go.id/direktori/index/pengadilan/pn-jakarta-utara/kategori/narkotika-dan-psikotropika-1.html"

# Create 'Putusan' folder if it doesn't exist
folder_name = "Putusan Narkotika dan Psikotropika"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Initialize a list to store the data
data_list = []

# List of keys to retrieve data
keys = [
    'Nomor',
    'Tingkat Proses',
    'Kata Kunci',
    'Tahun',
    'Tanggal Register',
    'Lembaga Peradilan',
    'Jenis Lembaga Peradilan',
    'Hakim Ketua',
    'Hakim Anggota',
    'Panitera',
    'Amar',
    'Catatan Amar',
    'Tanggal Musyawarah',
    'Tanggal Dibacakan'
]

# Start the crawling process
current_page = base_url
max_pages = 10  # Set the maximum number of pages to crawl
page_count = 0  # Initialize the page counter

while current_page and page_count < max_pages:
    print(f"Processing page: {current_page}")
    response = requests.get(current_page)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all matching decision links for Putusan PN Jakarta Utara
    links = soup.find_all('a', href=True)
    putusan_links = [link['href'] for link in links if link['href'].startswith('https://putusan3.mahkamahagung.go.id/direktori/putusan/')]

    # Fetch decision detail from each link
    for putusan_link in putusan_links:
        putusan_response = requests.get(putusan_link)
        putusan_soup = BeautifulSoup(putusan_response.text, 'html.parser')

        # Prepare to collect data
        data = {}
        try:
            # Extract required data
            for key in keys:
                element = putusan_soup.find(string=key)
                if element:
                    next_element = element.find_next()
                    if next_element:
                        data[key] = next_element.get_text(strip=True)
                    else:
                        data[key] = "Tidak ada data selanjutnya"
                else:
                    data[key] = "Key tidak ditemukan"

            # Get evidence from the amar section
            amar_element = putusan_soup.find(string="Catatan Amar")
            if amar_element:
                amar_content = amar_element.find_next().get_text(strip=True)
                bukti_start = amar_content.lower().find("barang bukti")
                if bukti_start != -1:
                    bukti_content = amar_content[bukti_start:]
                    data["Bukti"] = bukti_content
                else:
                    data["Bukti"] = "Bukti tidak ditemukan"
            else:
                data["Bukti"] = "Amar tidak ditemukan"

            data_list.append(data)

            # Download any PDF files
            pdf_links = putusan_soup.find_all('li')
            files_downloaded = False

            for li in pdf_links:
                a_tag = li.find('a', href=True)
                if a_tag and "download_file" in a_tag['href']:
                    file_url = a_tag['href']
                    print(f"Downloading: {file_url}")
                    file_response = requests.get(file_url)

                    sanitized_name = sanitize_filename(a_tag.text.strip())
                    file_path = os.path.join(folder_name, sanitized_name)

                    with open(file_path, 'wb') as file:
                      file.write(file_response.content)

                    files_downloaded = True

            if not files_downloaded:
                print("No PDF files found for download on this page.")

        except Exception as e:
            print(f"Error encountered while processing {putusan_link}: {e}")

    # Check for the "Next" page link using the updated HTML structure
    next_page_link = soup.find('a', rel='next')
    if next_page_link and next_page_link['href'] != '#':
        current_page = next_page_link['href']
    else:
        print("No next page found. Exiting loop.")
        break  # If no more pages, exit the loop

    page_count += 1  # Increment the page counter

# Create a DataFrame from the data collected
df = pd.DataFrame(data_list)
df.index += 1
df.to_csv('putusan_data_narkotika.csv')

print("Crawling completed!")

Processing page: https://putusan3.mahkamahagung.go.id/direktori/index/pengadilan/pn-jakarta-utara/kategori/narkotika-dan-psikotropika-1.html
Downloading: https://putusan3.mahkamahagung.go.id/direktori/download_file/2538c71f83c8ba73466ca549a5304c13/zip/zaefc2ca5970f85a9df8323131333031
Downloading: https://putusan3.mahkamahagung.go.id/direktori/download_file/2538c71f83c8ba73466ca549a5304c13/zip/zaefc2ca5970f85a9df8323131333031
Downloading: https://putusan3.mahkamahagung.go.id/direktori/download_file/2538c71f83c8ba73466ca549a5304c13/pdf/zaefc2ca5970f85a9df8323131333031
Downloading: https://putusan3.mahkamahagung.go.id/direktori/download_file/2538c71f83c8ba73466ca549a5304c13/pdf/zaefc2ca5970f85a9df8323131333031
Downloading: https://putusan3.mahkamahagung.go.id/direktori/download_file/a12a1683fd66a764c74a869ef870dc52/zip/zaefc2ca57be68269da0323131323538
Downloading: https://putusan3.mahkamahagung.go.id/direktori/download_file/a12a1683fd66a764c74a869ef870dc52/zip/zaefc2ca57be68269da032313132

Proses Penyimpanan pada Drive

In [ ]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Mengubah jalur sesuai kebutuhan
source = '/content/Putusan Narkotika dan Psikotropika'
destination = '/content/drive/My Drive/Putusan Narkotika'  # Ganti jalur dalam drive Anda di sini

# Menyalin folder ke Google Drive
shutil.copytree(source, destination)

print("Folder berhasil disalin ke Google Drive!")

Mounted at /content/drive
Folder berhasil disalin ke Google Drive!


In [ ]:
# prompt: print data ini https://drive.google.com/file/d/13leC1D0pFREBVIqifuoJQ2gfdfCm0UYN/view?usp=drive_link

import pandas as pd

pd = pd.read_csv('/content/drive/My Drive/Putusan Narkotika/putusan_data_narkotika.csv')

Error: File 'putusan_data_narkotika.csv' not found. Please run the data crawling script first.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Project NLP/putusan_data_narkotika.csv')
df

,Unnamed: 0,Nomor,Tingkat Proses,Kata Kunci,Tahun,Tanggal Register,Lembaga Peradilan,Jenis Lembaga Peradilan,Hakim Ketua,Hakim Anggota,Panitera,Amar,Catatan Amar,Tanggal Musyawarah,Tanggal Dibacakan,Bukti
0,1,948/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,23 Oktober 2024,PN JAKARTA UTARA,PN,Hakim Ketua Hasmy,"Br Hakim Anggota Dian Erdianto, Hakim Anggota...",Panitera Pengganti: Ari Palti Siregar,Lain-lain,Menyatakan Terdakwa IRisqi Fajar Ariantodan Te...,24 Desember 2024,24 Desember 2024,barang bukti berupa :1 (satu) bungkus plastik ...
1,2,893/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,8 Oktober 2024,PN JAKARTA UTARA,PN,Hakim Ketua Ranto Sabungan Silalahi,"Hakim Anggota Dian Erdianto, Br Hakim Anggota...",Panitera Pengganti: Ari Palti Siregar,Lain-lain,MENGADILI:Menyatakan Terdakwa ARIE SUSANTO Bin...,24 Desember 2024,24 Desember 2024,barang bukti berupa :1 (satu) bungkus plastic ...
2,3,904/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,10 Oktober 2024,PN JAKARTA UTARA,PN,Hakim Ketua Yuli Sinthesa Tristania,"Br Hakim Anggota Gede Sunarjana, Hakim Anggot...",Panitera Pengganti: Hendra Gunawan,Lain-lain,MENGADILI:Menyatakan Terdakwa IMUHAMAD SOLEH B...,19 Desember 2024,19 Desember 2024,barang bukti berupa :3 (tiga paket plastik kli...
3,4,885/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,7 Oktober 2024,PN JAKARTA UTARA,PN,Hakim Ketua Edi Junaedi,"Br Hakim Anggota Dian Erdianto, Hakim Anggota...",Panitera Pengganti Fitri Indriaty,Lain-lain,"MENGADILI:Menyatakan Terdakwa SONY SAPUTRA, te...",19 Desember 2024,19 Desember 2024,barang bukti :1 (satu) bungkus plastic klip be...
4,5,831/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,18 September 2024,PN JAKARTA UTARA,PN,Hakim Ketua Ranto Sabungan Silalahi,"Hakim Anggota Dian Erdianto, Br Hakim Anggota...",Panitera Pengganti Mellisa Harahap,Lain-lain,"IMenyatakan TerdakwaJhany Bin Sanda, telah ter...",19 Desember 2024,19 Desember 2024,barang bukti berupa :13 (tiga belas) paket nar...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,226,408/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,20 Mei 2024,PN JAKARTA UTARA,PN,Hakim Ketua Erly Soelistyarini,"Hakim Anggota R. Rudi Kindarto, Br Hakim Angg...",Panitera Pengganti: Sapto Suprio,Lain-lain,MENGADILIMenyatakan Terdakwa UNTUNG SUBAGYOter...,6 Agustus 2024,6 Agustus 2024,barang bukti berupa:8 (Delapan) bungkus plasti...
226,227,363/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,7 Mei 2024,PN JAKARTA UTARA,PN,Hakim Ketua Dian Erdianto,"Br Hakim Anggota Hotnar Simarmata, Hakim Angg...",Panitera Pengganti Budhyawan Kristianto.,Lain-lain,MENGADILI:Menyatakan Terdakwa Rudiyanto Bin Mi...,25 Juli 2024,25 Juli 2024,Barang bukti berupa :3 (tiga) paket narkotika ...
227,228,185/Pdt.G/2024/PA.Tkl,Pertama,Kewarisan,2024,18 Juli 2024,PA TAKALAR,PA,Hakim Ketua Hapsah,"S. Hi., Hakim Anggota Nur Akhriyani Zainal, B...",Panitera Pengganti Wahyuddin Wahid,Tidak Dapat Diterima,1. Menyatakan gugatan para Penggugat tidak dap...,31 Desember 2024,31 Desember 2024,Bukti tidak ditemukan
228,229,186/Pdt.P/2024/PN Unr,Pertama,Akta Kematian,2024,25 Nopember 2024,PN KAB SEMARANG,PN,Hakim Tunggal Raden Satya Adi Wicaksono,Hakim Tunggal Raden Satya Adi Wicaksono,Panitera Pengganti: Dina Pratiwi,Lain-lain,M E N E T A P K A N:Mengabulkan permohonan Pem...,18 Desember 2024,18 Desember 2024,Bukti tidak ditemukan


In [ ]:
df.drop(columns=['Unnamed: 0'], inplace=True)
df

,Nomor,Tingkat Proses,Kata Kunci,Tahun,Tanggal Register,Lembaga Peradilan,Jenis Lembaga Peradilan,Hakim Ketua,Hakim Anggota,Panitera,Amar,Catatan Amar,Tanggal Musyawarah,Tanggal Dibacakan,Bukti
0,948/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,23 Oktober 2024,PN JAKARTA UTARA,PN,Hakim Ketua Hasmy,"Br Hakim Anggota Dian Erdianto, Hakim Anggota...",Panitera Pengganti: Ari Palti Siregar,Lain-lain,Menyatakan Terdakwa IRisqi Fajar Ariantodan Te...,24 Desember 2024,24 Desember 2024,barang bukti berupa :1 (satu) bungkus plastik ...
1,893/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,8 Oktober 2024,PN JAKARTA UTARA,PN,Hakim Ketua Ranto Sabungan Silalahi,"Hakim Anggota Dian Erdianto, Br Hakim Anggota...",Panitera Pengganti: Ari Palti Siregar,Lain-lain,MENGADILI:Menyatakan Terdakwa ARIE SUSANTO Bin...,24 Desember 2024,24 Desember 2024,barang bukti berupa :1 (satu) bungkus plastic ...
2,904/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,10 Oktober 2024,PN JAKARTA UTARA,PN,Hakim Ketua Yuli Sinthesa Tristania,"Br Hakim Anggota Gede Sunarjana, Hakim Anggot...",Panitera Pengganti: Hendra Gunawan,Lain-lain,MENGADILI:Menyatakan Terdakwa IMUHAMAD SOLEH B...,19 Desember 2024,19 Desember 2024,barang bukti berupa :3 (tiga paket plastik kli...
3,885/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,7 Oktober 2024,PN JAKARTA UTARA,PN,Hakim Ketua Edi Junaedi,"Br Hakim Anggota Dian Erdianto, Hakim Anggota...",Panitera Pengganti Fitri Indriaty,Lain-lain,"MENGADILI:Menyatakan Terdakwa SONY SAPUTRA, te...",19 Desember 2024,19 Desember 2024,barang bukti :1 (satu) bungkus plastic klip be...
4,831/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,18 September 2024,PN JAKARTA UTARA,PN,Hakim Ketua Ranto Sabungan Silalahi,"Hakim Anggota Dian Erdianto, Br Hakim Anggota...",Panitera Pengganti Mellisa Harahap,Lain-lain,"IMenyatakan TerdakwaJhany Bin Sanda, telah ter...",19 Desember 2024,19 Desember 2024,barang bukti berupa :13 (tiga belas) paket nar...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,408/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,20 Mei 2024,PN JAKARTA UTARA,PN,Hakim Ketua Erly Soelistyarini,"Hakim Anggota R. Rudi Kindarto, Br Hakim Angg...",Panitera Pengganti: Sapto Suprio,Lain-lain,MENGADILIMenyatakan Terdakwa UNTUNG SUBAGYOter...,6 Agustus 2024,6 Agustus 2024,barang bukti berupa:8 (Delapan) bungkus plasti...
226,363/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,7 Mei 2024,PN JAKARTA UTARA,PN,Hakim Ketua Dian Erdianto,"Br Hakim Anggota Hotnar Simarmata, Hakim Angg...",Panitera Pengganti Budhyawan Kristianto.,Lain-lain,MENGADILI:Menyatakan Terdakwa Rudiyanto Bin Mi...,25 Juli 2024,25 Juli 2024,Barang bukti berupa :3 (tiga) paket narkotika ...
227,185/Pdt.G/2024/PA.Tkl,Pertama,Kewarisan,2024,18 Juli 2024,PA TAKALAR,PA,Hakim Ketua Hapsah,"S. Hi., Hakim Anggota Nur Akhriyani Zainal, B...",Panitera Pengganti Wahyuddin Wahid,Tidak Dapat Diterima,1. Menyatakan gugatan para Penggugat tidak dap...,31 Desember 2024,31 Desember 2024,Bukti tidak ditemukan
228,186/Pdt.P/2024/PN Unr,Pertama,Akta Kematian,2024,25 Nopember 2024,PN KAB SEMARANG,PN,Hakim Tunggal Raden Satya Adi Wicaksono,Hakim Tunggal Raden Satya Adi Wicaksono,Panitera Pengganti: Dina Pratiwi,Lain-lain,M E N E T A P K A N:Mengabulkan permohonan Pem...,18 Desember 2024,18 Desember 2024,Bukti tidak ditemukan


In [ ]:
df = df[df['Lembaga Peradilan'] == 'PN JAKARTA UTARA']
df

,Nomor,Tingkat Proses,Kata Kunci,Tahun,Tanggal Register,Lembaga Peradilan,Jenis Lembaga Peradilan,Hakim Ketua,Hakim Anggota,Panitera,Amar,Catatan Amar,Tanggal Musyawarah,Tanggal Dibacakan,Bukti
0,948/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,23 Oktober 2024,PN JAKARTA UTARA,PN,Hakim Ketua Hasmy,"Br Hakim Anggota Dian Erdianto, Hakim Anggota...",Panitera Pengganti: Ari Palti Siregar,Lain-lain,Menyatakan Terdakwa IRisqi Fajar Ariantodan Te...,24 Desember 2024,24 Desember 2024,barang bukti berupa :1 (satu) bungkus plastik ...
1,893/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,8 Oktober 2024,PN JAKARTA UTARA,PN,Hakim Ketua Ranto Sabungan Silalahi,"Hakim Anggota Dian Erdianto, Br Hakim Anggota...",Panitera Pengganti: Ari Palti Siregar,Lain-lain,MENGADILI:Menyatakan Terdakwa ARIE SUSANTO Bin...,24 Desember 2024,24 Desember 2024,barang bukti berupa :1 (satu) bungkus plastic ...
2,904/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,10 Oktober 2024,PN JAKARTA UTARA,PN,Hakim Ketua Yuli Sinthesa Tristania,"Br Hakim Anggota Gede Sunarjana, Hakim Anggot...",Panitera Pengganti: Hendra Gunawan,Lain-lain,MENGADILI:Menyatakan Terdakwa IMUHAMAD SOLEH B...,19 Desember 2024,19 Desember 2024,barang bukti berupa :3 (tiga paket plastik kli...
3,885/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,7 Oktober 2024,PN JAKARTA UTARA,PN,Hakim Ketua Edi Junaedi,"Br Hakim Anggota Dian Erdianto, Hakim Anggota...",Panitera Pengganti Fitri Indriaty,Lain-lain,"MENGADILI:Menyatakan Terdakwa SONY SAPUTRA, te...",19 Desember 2024,19 Desember 2024,barang bukti :1 (satu) bungkus plastic klip be...
4,831/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,18 September 2024,PN JAKARTA UTARA,PN,Hakim Ketua Ranto Sabungan Silalahi,"Hakim Anggota Dian Erdianto, Br Hakim Anggota...",Panitera Pengganti Mellisa Harahap,Lain-lain,"IMenyatakan TerdakwaJhany Bin Sanda, telah ter...",19 Desember 2024,19 Desember 2024,barang bukti berupa :13 (tiga belas) paket nar...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,463/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,10 Juni 2024,PN JAKARTA UTARA,PN,Hakim Ketua Gede Sunarjana,"Hakim Anggota Togi Pardede, M.hbr Hakim Anggo...",Panitera Pengganti: Asih Noviasari,Lain-lain,MENGADILI:Menyatakan terdakwa Terdakwa I RIZAL...,6 Agustus 2024,6 Agustus 2024,barang bukti berupa:1 (satu) bungkus plastic k...
223,589/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,8 Juli 2024,PN JAKARTA UTARA,PN,Hakim Ketua Slamet Widodo,"Hakim Anggota Iwan Irawan, Br Hakim Anggota ...",Panitera Pengganti Albert Costan Immanuel Sima...,Lain-lain,MENGADILI:Menyatakan terdakwa Stiawan Guna Had...,6 Agustus 2024,6 Agustus 2024,barang bukti :1 (satu) bungkus plastik yang be...
224,498/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,13 Juni 2024,PN JAKARTA UTARA,PN,Hakim Ketua Dian Erdianto,"Br Hakim Anggota Deny Riswanto, Hakim Anggota...",Panitera Pengganti: Ari Palti Siregar,Lain-lain,"Menyatakan TerdakwaArif Bin Alm. Tabroni, tela...",6 Agustus 2024,6 Agustus 2024,barang bukti berupa :3 (tiga) bungkus plastic ...
225,408/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,20 Mei 2024,PN JAKARTA UTARA,PN,Hakim Ketua Erly Soelistyarini,"Hakim Anggota R. Rudi Kindarto, Br Hakim Angg...",Panitera Pengganti: Sapto Suprio,Lain-lain,MENGADILIMenyatakan Terdakwa UNTUNG SUBAGYOter...,6 Agustus 2024,6 Agustus 2024,barang bukti berupa:8 (Delapan) bungkus plasti...


In [ ]:
df.insert(0, 'No', range(1, len(df) + 1))
df

,No,Nomor,Tingkat Proses,Kata Kunci,Tahun,Tanggal Register,Lembaga Peradilan,Jenis Lembaga Peradilan,Hakim Ketua,Hakim Anggota,Panitera,Amar,Catatan Amar,Tanggal Musyawarah,Tanggal Dibacakan,Bukti
0,1,948/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,23 Oktober 2024,PN JAKARTA UTARA,PN,Hakim Ketua Hasmy,"Br Hakim Anggota Dian Erdianto, Hakim Anggota...",Panitera Pengganti: Ari Palti Siregar,Lain-lain,Menyatakan Terdakwa IRisqi Fajar Ariantodan Te...,24 Desember 2024,24 Desember 2024,barang bukti berupa :1 (satu) bungkus plastik ...
1,2,893/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,8 Oktober 2024,PN JAKARTA UTARA,PN,Hakim Ketua Ranto Sabungan Silalahi,"Hakim Anggota Dian Erdianto, Br Hakim Anggota...",Panitera Pengganti: Ari Palti Siregar,Lain-lain,MENGADILI:Menyatakan Terdakwa ARIE SUSANTO Bin...,24 Desember 2024,24 Desember 2024,barang bukti berupa :1 (satu) bungkus plastic ...
2,3,904/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,10 Oktober 2024,PN JAKARTA UTARA,PN,Hakim Ketua Yuli Sinthesa Tristania,"Br Hakim Anggota Gede Sunarjana, Hakim Anggot...",Panitera Pengganti: Hendra Gunawan,Lain-lain,MENGADILI:Menyatakan Terdakwa IMUHAMAD SOLEH B...,19 Desember 2024,19 Desember 2024,barang bukti berupa :3 (tiga paket plastik kli...
3,4,885/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,7 Oktober 2024,PN JAKARTA UTARA,PN,Hakim Ketua Edi Junaedi,"Br Hakim Anggota Dian Erdianto, Hakim Anggota...",Panitera Pengganti Fitri Indriaty,Lain-lain,"MENGADILI:Menyatakan Terdakwa SONY SAPUTRA, te...",19 Desember 2024,19 Desember 2024,barang bukti :1 (satu) bungkus plastic klip be...
4,5,831/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,18 September 2024,PN JAKARTA UTARA,PN,Hakim Ketua Ranto Sabungan Silalahi,"Hakim Anggota Dian Erdianto, Br Hakim Anggota...",Panitera Pengganti Mellisa Harahap,Lain-lain,"IMenyatakan TerdakwaJhany Bin Sanda, telah ter...",19 Desember 2024,19 Desember 2024,barang bukti berupa :13 (tiga belas) paket nar...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,196,463/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,10 Juni 2024,PN JAKARTA UTARA,PN,Hakim Ketua Gede Sunarjana,"Hakim Anggota Togi Pardede, M.hbr Hakim Anggo...",Panitera Pengganti: Asih Noviasari,Lain-lain,MENGADILI:Menyatakan terdakwa Terdakwa I RIZAL...,6 Agustus 2024,6 Agustus 2024,barang bukti berupa:1 (satu) bungkus plastic k...
223,197,589/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,8 Juli 2024,PN JAKARTA UTARA,PN,Hakim Ketua Slamet Widodo,"Hakim Anggota Iwan Irawan, Br Hakim Anggota ...",Panitera Pengganti Albert Costan Immanuel Sima...,Lain-lain,MENGADILI:Menyatakan terdakwa Stiawan Guna Had...,6 Agustus 2024,6 Agustus 2024,barang bukti :1 (satu) bungkus plastik yang be...
224,198,498/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,13 Juni 2024,PN JAKARTA UTARA,PN,Hakim Ketua Dian Erdianto,"Br Hakim Anggota Deny Riswanto, Hakim Anggota...",Panitera Pengganti: Ari Palti Siregar,Lain-lain,"Menyatakan TerdakwaArif Bin Alm. Tabroni, tela...",6 Agustus 2024,6 Agustus 2024,barang bukti berupa :3 (tiga) bungkus plastic ...
225,199,408/Pid.Sus/2024/PN Jkt.Utr,Pertama,Narkotika,2024,20 Mei 2024,PN JAKARTA UTARA,PN,Hakim Ketua Erly Soelistyarini,"Hakim Anggota R. Rudi Kindarto, Br Hakim Angg...",Panitera Pengganti: Sapto Suprio,Lain-lain,MENGADILIMenyatakan Terdakwa UNTUNG SUBAGYOter...,6 Agustus 2024,6 Agustus 2024,barang bukti berupa:8 (Delapan) bungkus plasti...


In [ ]:
df.to_csv('putusan_data_narkotika.csv', index=False)